En este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

In [151]:
import pandas as pd
import numpy as np
import sidetable
import re

In [152]:
df=pd.read_csv('../EDA/datos/attack_limpio.csv', index_col=0)

In [153]:
df.head(2)

,case_number,year,country,age,time,species_,mes,fatal_,sex2
0,1800.00.00,1800,seychelles,NaN,NaN,NaN,NaN,y,F
1,1797.05.28.R,1797,NaN,NaN,NaN,NaN,May,y,NaN


1. Columna de species: 
   
   Si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.
   - Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías
    que hemos definido previamente.
         
        *imaginemos que el valor de una celda es el siguiente

                    'White shark, 3.5 m'

        *tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
        Un patrón que podriamos usar es: 

                    patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

        *de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

    - Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [109]:
patronblanco= '.*([W|w]hite|HITE).*'

patrontigre= '.*([T|t]iger|IGER).*'

patronbull= '.*([B|b]ull|ULL).*'

patrongrey= '.*([G|g]rey|REY).*'

patronlemon= '.*([L|l]emon|LEMON).*'


In [127]:
try:
    if df[df['species_'].str.contains('NaN', regex=True )]:
        df['cleaned_species']= 'Unspecified'
    if df[df['species_'].str.contains(patronblanco, regex=True )]:
        df['cleaned_species']= 'White'
except:
    pass

In [116]:
def tiburon(patron, val):

    if val.str.contains(patronblanco):
        df['cleaned_species']= 'White'
    elif val.str.contains(patrontigre,val):
        df['cleaned_species']= 'Tigre'   
    elif re.findall(patronbull,val):
        df['cleaned_species']= 'Bull'
    elif re.findall(patrongrey,val):
        df['cleaned_species']= 'Grey'
    elif re.findall(patronlemon,val):
        df['cleaned_species']= 'Lemon'
    else:
         df['cleaned_species']= 'Unspecified'   

TypeError: expected string or bytes-like object

In [143]:
def tiburon(df, patron):
    try:
        
        return re.findall(patron, df)[0] #Le pasamos la posición cero porque refindall nos busca por una lista.

    except:
        pass

In [144]:
df['cleaned_species']=tiburon(df, patronblanco)
df['cleaned_species']=tiburon(df['species_'], patrontigre)
df['cleaned_species']=tiburon(df['species_'], patronbull)
df['cleaned_species']=tiburon(df['species_'], patrongrey)
df['cleaned_species']=tiburon(df['species_'], patronlemon)

In [154]:
df.sample(10)

,case_number,year,country,age,time,species_,mes,fatal_,sex2
5623,1906.09.27.R.a,1906,indonesia,NaN,NaN,NaN,Sep,y,M
217,2018.01.28,2018,australia,18,08h00,Bull shark,Jan,n,M
5086,1934.12.23.a,1934,australia,12,13h00,NaN,Oct,n,M
4937,1940.00.00.c,1940,south africa,NaN,NaN,Questionable incident,NaN,NaN,NaN
1512,2007.08.15,2007,usa,20,Night,Possibly a 6' bull shark,Aug,n,M
4423,1956.00.00.h,1956,papua new guinea,NaN,NaN,NaN,NaN,n,M
2650,1993.07.07,1993,usa,12,18h12,"1.8 m to 2.4 m [6' to 8'] shark, tooth fragmen...",Jul,n,F
2071,2001.09.03.a,2001,usa,23,18h00,A large white shark was filmed by divers on a ...,Sep,n,F
720,2014.05.23,2014,france,NaN,Morning,3m shark,May,n,M
4629,1950.07.00,1950,usa,21,NaN,0.7 m [2.5'] shark,Jul,n,M


2. Columna de age: 
   
   es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
   - Edad en formato string
   - Edades separadas por &, or, to, >
   - Edades con ?
   Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
   - Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
   - Puede que os salga un error similar a este:
  
                     TypeError: expected string or bytes-like object
                  
     Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

                     df['nombre_columna'] = df['nombre_columna'].astype(str)
                     
   - Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. 
   - Por último cambiad el tipo de la columna de string a integer.
  


3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.